# Assignment1. Multiclass SVM 구현


과제에 필요한 기본적 패키지와 데이터를 불러온다.

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

#IRIS 데이터 로드
iris =  sns.load_dataset('iris') 
X= iris.iloc[:,:4] #학습할데이터
y = iris.iloc[:,-1] #타겟
print(y)

0         setosa
1         setosa
2         setosa
3         setosa
4         setosa
         ...    
145    virginica
146    virginica
147    virginica
148    virginica
149    virginica
Name: species, Length: 150, dtype: object


데이터는 우리가 아는 iris 데이터이다. 이후 svm모델링을 진행하기 위해서 train과 test set으로 데이터를 분리해준다. 여기서는 test size를 20%로 진행했다.

In [3]:
from sklearn.model_selection import train_test_split #test/train 데이터로 분리
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=48)

In [4]:
y_train

110     virginica
69     versicolor
148     virginica
39         setosa
53     versicolor
          ...    
64     versicolor
91     versicolor
81     versicolor
51     versicolor
0          setosa
Name: species, Length: 120, dtype: object

원할한 분석을 위하여 scaler를 통해 데이터 전처리를 진행한다. scaler는 standardscaler를 사용했다. train데이터에 대해서는 fit_transform을, test 데이터에는 transform만을 진행한다

In [5]:
scal = StandardScaler() #scaling
X_train = scal.fit_transform(X_train)
X_test = scal.transform(X_test)

In [6]:
## Case 1 : One vs Rest SVM을 이부분에 구현해주세요 위 결과들을 이용해서 multi class SVM을 직접 구현해주세요! 하드코딩이 하시기 편할겁니다.

def label(y_train):
    y_train = pd.get_dummies(y_train)
    return y_train

def machine(X_train, y_train, c = 5, g = 5):
    svcs = []
    y_train = label(y_train)
    
    for i in range(len(y_train.columns)):
        svc = SVC(kernel = 'rbf', C = c, gamma = g)
        svc.fit(X_train,y_train.iloc[:,i])
        svcs.append(svc)
        
    return svcs

def predict(X_test):
    
    df = pd.DataFrame(index=range(0,len(X_test)))
    list_ = []
    list_2 = []
    
    for i in range(len(machine(X_train, y_train, c = 5, g = 5))):
        svc = pd.DataFrame(machine(X_train, y_train, c = 5, g = 5)[i].decision_function(X_test))
        df = pd.concat([df,svc],axis = 1)
    
    df_list = df.values.tolist()
    
    for j in range(len(df_list)):
        number = np.argmax(df_list[j])
        list_.append(number) 
    
    for k in range(len(list_)):
        if list_[k] == 0:
            list_2.append(label(y_test).columns[0])
        elif list_[k] == 1:
            list_2.append(label(y_test).columns[1])
        elif list_[k] == 2:
            list_2.append(label(y_test).columns[2])
                
    return list_2    

def accuracy(y_test):
    from sklearn.metrics import accuracy_score
    print (accuracy_score(y_test,predict(X_test)))

본격적으로 위의 방식으로 one vs rest svm을 만들어준다. 가장 근본이 되는 아이디어는 한 분류기는 하나의 class에 대해서만 구분한다는 점이다. 즉, 우리가 사용하는 데이터를 바탕으로 말하자면, versicolor을 구분하는 분류기는 versicolor가 맞는지 아닌지만을 구분한다는 점이다. 

이를 가능하게 하기 위해서, 범주형 데이터를 더비 변수로 만들어주는 label 함수를 만들어준다. 그리고 machine 함수를 통해 각 label의 class가 맞는지 아닌지를 구분해주는 svm을 만들어준다. for 함수를 통해 만들어진 분류기에 train 데이터를 학습시킨다. 각 분류기들을 리스트 형식으로 return해준다. c,g 값을 5로 설정한 이유는 주어진 과제에서 각각 5라고 제시했기 때문에 이를 따랐다.

predict 함수는 위와 같다. 우선 빈 데이터 프레임과 빈 리스트 2개를 만들어준다. 첫번째 for문을 통해 각 분류기의 decision funcion을 데이터 프레임으로 만들어주고, 이를 모두 합쳐 한 행이 각각의 분류기를 통해 얻은 hyperplane과의 거리를 나타내는 데이터 프레임을 만든다. 이 후, 이것을 list로 만들어 앞으로의 계산을 용이하게 한다.
(굳이 decision function을 사용한 이유는 sklearn의 문서를 참고했기 때문이다. 자세한 내용은 아래의 링크에 argmax를 찾아 읽어보길 바란다.)
https://scikit-learn.org/stable/modules/svm.html

거리를 나타내는 list를 argmax 함수를 사용해 가장 큰 값의 인덱스 값이 무엇인지 파악하고 이를 list_에 더해준다. 이후 마지막 for문을 돌려 0일때, 1일때, 2일때 어떤 품종인지를 직접 선언한다. 그리고 이 값을 list_2에 받아준다. 마지막으로 list_2를 return 해주면 분류기를 통해 예측한 값이 나올 것이다.

마지막으로 sklearn의 accuracy_score를 사용해, 정확도를 측정할 수 있는 accuracy 함수로 만들어준다. 

In [7]:
machine(X_train, y_train, c = 5, g = 5)

[SVC(C=5, gamma=5), SVC(C=5, gamma=5), SVC(C=5, gamma=5)]

In [8]:
predict(X_test)

['versicolor',
 'versicolor',
 'versicolor',
 'virginica',
 'virginica',
 'virginica',
 'setosa',
 'virginica',
 'setosa',
 'versicolor',
 'virginica',
 'setosa',
 'setosa',
 'virginica',
 'versicolor',
 'versicolor',
 'setosa',
 'versicolor',
 'virginica',
 'virginica',
 'setosa',
 'virginica',
 'versicolor',
 'versicolor',
 'virginica',
 'setosa',
 'setosa',
 'virginica',
 'virginica',
 'versicolor']

In [9]:
accuracy(y_test)

0.8666666666666667


각 함수를 돌린 값들은 위와 같다. 최종적으로 accuracy(y_test)의 값을 보면 약 0.867의 값이 나왔다. 이 값은 sklearn이 제공하는 svm의 결과와 동일한 것을 확인할 수 있다.

In [11]:
# 원래 라이브러리가 제공하는 multi class SVM과 여러분이 구현한 multiclass SVM 결과를 비교해주세요
from sklearn.model_selection import train_test_split #데이터셋 분리
from sklearn import metrics
X_train_2, X_test_2, y_train_2, y_test_2 = train_test_split(X, y, test_size=0.2, random_state=48)

svm_4 = SVC(kernel ='rbf', C = 5, gamma = 5)
svm_4.fit(X_train_2, y_train_2)
y_pred = svm_4.predict(X_test_2)

metrics.accuracy_score(y_test_2,y_pred)

0.8666666666666667